#Tono y tema sin datos de entrenamiento

##Tono

In [ ]:
# @title Instalar librerías
# Instalar librerías necesarias de forma silenciosa
!pip install -qq sentence-transformers pandas openpyxl tqdm emoji

# Importar librerías
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from tqdm.notebook import tqdm
import torch
import emoji
import re
from collections import defaultdict
import unicodedata
from google.colab import userdata # Import userdata to access secrets if needed for other models

# Verificar y configurar el uso de GPU
if torch.cuda.is_available():
    device = 'cuda'
    print("Using GPU for computation.")
else:
    device = 'cpu'
    print("Using CPU for computation.")

# Definir las categorías de sentimiento y sus palabras clave
# Estas palabras clave son descriptivas y ayudan al modelo a entender el 'centro' de cada sentimiento.
# Sin embargo, el modelo ya está pre-entrenado para entender el sentimiento directamente.
sentiment_keywords = {
    "Positivo": [
        "excelente", "bueno", "genial", "positivo", "fantástico", "maravilloso",
        "feliz", "contento", "agradable", "éxito", "perfecto", "gran", "magnífico",
        "óptimo", "mejor", "acuerdo", "aprobación", "felicitaciones", "elogio"
    ],
    "Neutro": [
        "neutral", "información", "hecho", "dato", "reporte", "observación",
        "análisis", "declaración", "noticia", "comunicado", "menciona", "describe",
        "sin emoción", "objetivo", "estado", "situación", "acerca de", "sobre"
    ],
    "Negativo": [
        "malo", "terrible", "negativo", "problema", "crisis", "difícil", "triste",
        "decepcionante", "preocupante", "fallo", "error", "desastre", "crítica",
        "queja", "inconformidad", "pésimo", "peor", "desacuerdo", "rechazo", "malestar"
    ]
}

# No verificamos palabras clave duplicadas para sentimiento ya que las superposiciones son más comunes
# y el modelo se basa más en el significado contextual.

# Cargar textos a clasificar
from google.colab import files
print("Sube el archivo con los textos a clasificar:")
uploaded_texts = files.upload()

texts_file_name = list(uploaded_texts.keys())[0]
texts_df = pd.read_excel(texts_file_name)

if 'resumen' not in texts_df.columns:
    raise ValueError("El archivo a clasificar debe contener una columna 'resumen'.")

# Cargar el modelo de Sentence-BERT específico para análisis de sentimiento en español
# 'pysentimiento/robertuito-sentiment-analysis' es un modelo robusto y público para español.
# Este modelo NO requiere token de Hugging Face.
print(f"Loading sentiment analysis model to {device}...")
sentiment_model = SentenceTransformer('pysentimiento/robertuito-sentiment-analysis', device=device)
print("Sentiment analysis model loaded successfully.")

# Crear embeddings de las categorías de sentimiento
# Aunque el modelo ya está pre-entrenado, usar estos embeddings como 'prototipos'
# ayuda a guiar la clasificación por similitud cosenoidal.
sentiment_embeddings = {}
for sentiment, keywords in sentiment_keywords.items():
    embeddings = sentiment_model.encode(keywords, convert_to_tensor=True)
    sentiment_embedding = embeddings.mean(dim=0)
    sentiment_embeddings[sentiment] = sentiment_embedding.to(device)

# Apilar todos los embeddings de sentimiento en un solo tensor
sentiment_embeddings_tensor = torch.stack(list(sentiment_embeddings.values()))
sentiment_names = list(sentiment_embeddings.keys())

# Función para normalizar el texto (la misma que antes)
def normalize_text(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text

# Función para preprocesar el texto (la misma que antes)
def preprocess_text(text):
    text = normalize_text(text)
    # Eliminar URLs
    text = re.sub(r'http\S+', '', text)
    # Eliminar caracteres especiales (excepto hashtags si fueran relevantes, pero para sentimiento no suelen serlo)
    text = re.sub(r'[^\w\s]', '', text) # Simplificado para sentimiento
    return text

# Función de clasificación de sentimiento
# Ahora usamos un enfoque de 'coincidencia estricta' para palabras clave de sentimiento
# y luego la similitud del modelo para el resto.
def classify_sentiment(text, model, sentiment_embeddings_tensor, sentiment_names, threshold=0.6, min_score_diff=0.1):
    text_preprocessed = preprocess_text(text)

    # 1. Búsqueda de coincidencia exacta de palabras clave de sentimiento (alta confianza)
    # Esta es una heurística fuerte para asegurar que "excelente" siempre sea Positivo.
    for sentiment, keywords in sentiment_keywords.items():
        for kw in keywords:
            if normalize_text(kw) in text_preprocessed.split(): # Split para asegurar palabra completa
                return sentiment, 1.0

    # 2. Clasificación por similitud con embeddings de categorías (si no hay coincidencia exacta)
    text_embedding = model.encode(text_preprocessed, convert_to_tensor=True).to(device)
    cos_similarities = util.cos_sim(text_embedding, sentiment_embeddings_tensor)[0]

    top_idx = torch.argmax(cos_similarities).item()
    top_sentiment = sentiment_names[top_idx]
    top_score = cos_similarities[top_idx].item()

    # Evaluar la diferencia con la segunda mejor puntuación para mayor precisión
    sorted_scores, _ = torch.sort(cos_similarities, descending=True)
    if len(sorted_scores) > 1:
        second_score = sorted_scores[1].item()
        score_diff = top_score - second_score
    else:
        score_diff = top_score # Si solo hay una categoría, la diferencia es la propia puntuación

    # Aplicar umbrales para decidir la confianza o si es "Neutro" por ambigüedad
    if top_score >= threshold and score_diff >= min_score_diff:
        return top_sentiment, top_score
    else:
        # Si la confianza no es alta o la diferencia es baja,
        # puede indicar que el texto es más neutro o ambiguo.
        # Podríamos forzar a "Neutro" o dejar la categoría de mayor similitud
        # con una confianza menor. Optamos por dejar la de mayor similitud.
        return top_sentiment, top_score


# Clasificar textos y almacenar resultados
texts = texts_df['resumen'].fillna("").tolist()
results = []

# Definir umbrales de confianza para el análisis de sentimiento
sentiment_threshold = 0.65  # Umbral de confianza para la clasificación de sentimiento
sentiment_min_score_diff = 0.15 # Diferencia mínima entre top 1 y top 2 para clasificación clara

# Procesamiento por lotes
batch_size = 32 # Ajusta este valor según la memoria de la GPU
num_batches = (len(texts) + batch_size - 1) // batch_size

for i in tqdm(range(num_batches), desc="Clasificando sentimiento por lotes"):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(texts))
    batch_texts = texts[start_idx:end_idx]

    # Preprocesar textos del lote (directamente aquí para evitar re-preprocesar en la función)
    batch_preprocessed_texts = [preprocess_text(text) for text in batch_texts]

    # Codificar todos los textos del lote a la vez para eficiencia en GPU
    # Nota: No pasamos el token aquí, ya que 'pysentimiento/robertuito-sentiment-analysis' es público
    batch_text_embeddings = sentiment_model.encode(batch_preprocessed_texts, convert_to_tensor=True).to(device)

    for j, original_text in enumerate(batch_texts):
        # Obtener el embedding pre-calculado para el texto actual del lote
        current_text_embedding = batch_text_embeddings[j]
        current_text_preprocessed = batch_preprocessed_texts[j]

        # 1. Coincidencia exacta de palabras clave de sentimiento (prioridad alta)
        found_exact_match = False
        for sentiment, keywords in sentiment_keywords.items():
            for kw in keywords:
                # Usar .split() para asegurar que 'bueno' en 'muy bueno' no coincida con 'buen'
                if normalize_text(kw) in current_text_preprocessed.split():
                    results.append({
                        "texto": original_text,
                        "sentimiento": sentiment,
                        "confianza": 1.0 # Confianza máxima si hay coincidencia exacta
                    })
                    found_exact_match = True
                    break
            if found_exact_match:
                break

        if found_exact_match:
            continue # Si ya clasificamos por coincidencia exacta, pasamos al siguiente texto

        # 2. Clasificación por similitud cosenoidal (para el resto de textos)
        cos_similarities = util.cos_sim(current_text_embedding, sentiment_embeddings_tensor)[0]

        top_idx = torch.argmax(cos_similarities).item()
        top_sentiment = sentiment_names[top_idx]
        top_score = cos_similarities[top_idx].item()

        # Obtener la segunda mayor similitud para evaluar la diferencia
        sorted_scores, _ = torch.sort(cos_similarities, descending=True)
        if len(sorted_scores) > 1:
            second_score = sorted_scores[1].item()
            score_diff = top_score - second_score
        else:
            score_diff = top_score # Si solo hay una categoría, la diferencia es la puntuación misma

        # Decidir la categoría basada en el umbral y la diferencia de puntuaciones
        if top_score >= sentiment_threshold and score_diff >= sentiment_min_score_diff:
            final_sentiment = top_sentiment
            final_confidence = top_score
        else:
            # Si la confianza no es lo suficientemente alta o hay mucha ambigüedad,
            # lo asignamos como Neutro para ser más conservadores, o la de mayor similitud
            # con su score.
            # Aquí, priorizamos la categoría de mayor similitud si no cumple los umbrales
            # de "alta confianza" pero aún así necesitamos una clasificación.
            final_sentiment = top_sentiment # O podrías forzar a "Neutro" aquí si prefieres.
            final_confidence = top_score

        results.append({
            "texto": original_text,
            "sentimiento": final_sentiment,
            "confianza": final_confidence
        })

# Exportar resultados
results_df = pd.DataFrame(results)
results_df.to_excel("sentiment_classification_results.xlsx", index=False)
print("Clasificación de sentimiento completada. Descarga el archivo:")
files.download("sentiment_classification_results.xlsx")

##Tema

In [ ]:
# @title Tema
# Instalar librerías necesarias de forma silenciosa
!pip install -qq sentence-transformers pandas openpyxl tqdm emoji

# Importar librerías
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from tqdm.notebook import tqdm
import torch
import emoji
import re
from collections import defaultdict
import unicodedata
from google.colab import userdata # Import userdata to access secrets (still good practice for other needs)

try:
    hf_token = userdata.get('HF_TOKEN')
    if hf_token is None:
        print("HF_TOKEN secret not found. This is okay for the chosen public model.")
    else:
        print("Hugging Face token accessed from Colab Secrets (not strictly needed for this model).")
except Exception as e:
    print(f"Error accessing HF_TOKEN secret: {e}. This is okay for the chosen public model.")
    hf_token = None


# Verificar uso de GPU
if torch.cuda.is_available():
    device = 'cuda'
    print("Using GPU for computation.")
else:
    device = 'cpu'
    print("Using CPU for computation.")

# Definir 10 temas muy genéricos para análisis de contenido
predefined_keywords = {
    "Noticias y Eventos Actuales": [
        "noticia", "actualidad", "evento", "suceso", "última hora", "acontecimiento",
        "incidente", "desarrollo", "informe", "periodismo", "reportaje", "cobertura",
        "reciente", "hoy", "ayer"
    ],
    "Opiniones y Debates": [
        "opinión", "debate", "discusión", "punto de vista", "perspectiva", "argumento",
        "controversia", "polémica", "crítica", "elogio", "comentario", "sentimiento",
        "postura", "análisis", "reflexión"
    ],
    "Productos y Servicios": [
        "producto", "servicio", "marca", "empresa", "oferta", "lanzamiento",
        "característica", "función", "precio", "disponibilidad", "calidad",
        "experiencia", "uso", "beneficio", "solución"
    ],
    "Economía y Negocios": [
        "economía", "negocio", "mercado", "inversión", "finanzas", "empresa",
        "crecimiento", "crisis económica", "sector", "comercio", "desempleo",
        "tendencia económica", "capital", "ingresos", "gastos"
    ],
    "Política y Gobierno": [
        "política", "gobierno", "ley", "regulación", "decisión", "elecciones",
        "partido", "líder", "democracia", "justicia", "reforma", "público",
        "estado", "norma", "oficial"
    ],
    "Sociedad y Cultura": [
        "sociedad", "cultura", "comunidad", "educación", "salud", "arte",
        "historia", "valores", "tendencia social", "grupo", "tradición",
        "identidad", "cambio social", "bienestar", "equidad"
    ],
    "Tecnología e Innovación": [
        "tecnología", "innovación", "digital", "internet", "software", "hardware",
        "futuro", "desarrollo", "descubrimiento", "aplicación", "sistema",
        "robótica", "inteligencia artificial", "conectividad", "ciberseguridad"
    ],
    "Medio Ambiente y Sostenibilidad": [
        "medio ambiente", "sostenibilidad", "cambio climático", "ecología",
        "contaminación", "recursos naturales", "biodiversidad", "energía",
        "conservación", "reciclaje", "impacto ambiental", "verde", "planeta",
        "residuos"
    ],
    "Salud y Bienestar": [
        "salud", "bienestar", "enfermedad", "tratamiento", "medicina", "hospital",
        "cuidado", "prevención", "alimentación", "ejercicio", "salud mental",
        "diagnóstico", "síntoma", "recuperación", "terapia"
    ],
    "Deportes y Entretenimiento": [
        "deporte", "entretenimiento", "película", "música", "serie", "juego",
        "artista", "celebridad", "evento deportivo", "cultura pop", "diversión",
        "espectáculo", "campeonato", "afición", "creativo"
    ]
}

# Verificar que no haya palabras clave duplicadas entre categorías
keyword_to_topics = defaultdict(list)
for topic, keywords in predefined_keywords.items():
    for kw in keywords:
        keyword_to_topics[kw.lower()].append(topic)

# Encontrar palabras clave que se repiten en múltiples categorías
duplicate_keywords = {kw: topics for kw, topics in keyword_to_topics.items() if len(topics) > 1}

if duplicate_keywords:
    print("Palabras clave duplicadas encontradas:")
    for kw, topics in duplicate_keywords.items():
        print(f"'{kw}' se encuentra en las categorías: {topics}")
else:
    print("No se encontraron palabras clave duplicadas entre categorías.")

# Cargar textos a clasificar
from google.colab import files
print("Sube el archivo con los textos a clasificar:")
uploaded_texts = files.upload()

texts_file_name = list(uploaded_texts.keys())[0]
texts_df = pd.read_excel(texts_file_name)

if 'resumen' not in texts_df.columns:
    raise ValueError("El archivo a clasificar debe contener una columna 'resumen'.")

# Cargar el modelo de Sentence-BERT (AHORA USANDO UN MODELO PÚBLICO Y ACCESIBLE)
embedding_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device=device) # Removed 'token=hf_token'

# Crear embeddings de temas
topic_embeddings = {}
for topic, keywords in predefined_keywords.items():
    if keywords:
        embeddings = embedding_model.encode(keywords, convert_to_tensor=True)
        topic_embedding = embeddings.mean(dim=0)
    else:
        topic_embedding = torch.zeros(embedding_model.get_sentence_embedding_dimension())
    topic_embeddings[topic] = topic_embedding.to(device)

# Apilar todos los embeddings de temas en un solo tensor
topic_embeddings_tensor = torch.stack(list(topic_embeddings.values()))
topic_names = list(topic_embeddings.keys())

# Función para normalizar el texto
def normalize_text(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text

# Función para preprocesar el texto
def preprocess_text(text):
    text = normalize_text(text)
    # Eliminar URLs
    text = re.sub(r'http\S+', '', text)
    # Eliminar caracteres especiales excepto hashtags
    text = re.sub(r'[^\w#\s]', '', text)
    return text

# Función para verificar coincidencia exacta de palabras clave
def exact_match_all(text, predefined_keywords):
    matched_categories = []
    # Tokenizar el texto para incluir palabras y hashtags
    tokens = re.findall(r'#\w+|\w+', text)
    # Normalizar tokens
    normalized_tokens = [normalize_text(tok) for tok in tokens]

    for topic, keywords in predefined_keywords.items():
        for kw in keywords:
            kw_normalized = normalize_text(kw)
            if kw_normalized in normalized_tokens:
                matched_categories.append(topic)
                break
    return matched_categories

# Clasificar textos
texts = texts_df['resumen'].fillna("").tolist()
results = []

# Definir umbral de confianza y diferencia mínima para clasificación
threshold = 0.68  # Mantener un umbral alto para precisión
min_score_diff = 0.15 # Mantener una buena diferencia para desempate claro

# Procesamiento por lotes
batch_size = 128 # Ajusta este valor según la memoria de la GPU
num_batches = (len(texts) + batch_size - 1) // batch_size

for i in tqdm(range(num_batches), desc="Clasificando textos por lotes"):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(texts))
    batch_texts = texts[start_idx:end_idx]

    batch_preprocessed_texts = [preprocess_text(text) for text in batch_texts]

    # Codificar todos los textos del lote a la vez
    batch_embeddings = embedding_model.encode(batch_preprocessed_texts, convert_to_tensor=True).to(device)

    for j, original_text in enumerate(batch_texts):
        text_preprocessed = batch_preprocessed_texts[j]
        text_embedding = batch_embeddings[j]

        # Buscar coincidencias de palabras clave primero
        matched_categories = exact_match_all(text_preprocessed, predefined_keywords)

        if matched_categories:
            from collections import Counter
            category_counts = Counter(matched_categories)
            top_category = category_counts.most_common(1)[0][0]
            results.append({
                "texto": original_text,
                "categoria": top_category,
                "confianza": 1.0  # Confianza alta si hay coincidencia exacta
            })
        else:
            # Calcular similitudes con todos los temas
            cos_similarities = util.cos_sim(text_embedding, topic_embeddings_tensor)[0]

            # Obtener el índice del tema con mayor similitud
            top_idx = torch.argmax(cos_similarities).item()
            top_topic = topic_names[top_idx]
            top_score = cos_similarities[top_idx].item()

            # Obtener la segunda mayor similitud para evaluar la diferencia
            sorted_scores, sorted_indices = torch.sort(cos_similarities, descending=True)
            if len(sorted_scores) > 1:
                second_score = sorted_scores[1].item()
                score_diff = top_score - second_score
            else:
                score_diff = top_score # Si solo hay un tema, la diferencia es la puntuación misma

            # Decidir la categoría basada en el umbral y la diferencia de puntuaciones
            if top_score >= threshold and score_diff >= min_score_diff:
                categoria = top_topic
                confianza = top_score
            else:
                # Si no cumple los criterios de confianza, se asigna a la categoría de mayor similitud
                # pero la confianza reflejará que la correspondencia no fue muy fuerte.
                categoria = top_topic
                confianza = top_score

            results.append({
                "texto": original_text,
                "categoria": categoria,
                "confianza": confianza
            })

# Exportar resultados
results_df = pd.DataFrame(results)
results_df.to_excel("classification_results.xlsx", index=False)
print("Clasificación completada. Descarga el archivo:")
files.download("classification_results.xlsx")

---

#Tono y tema con datos de entrenamiento

##Tono

In [ ]:
# @title Instalación
!pip install -qq sentiment-analysis-spanish scikit-learn

In [ ]:
# @title Subir excel de entrenamiento
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from google.colab import files

def cargar_y_preprocesar_datos(archivo):
    df = pd.read_excel(archivo)

    # Mapear etiquetas de texto a valores numéricos
    label_mapping = {'Positivo': 1, 'Neutro': 0, 'Negativo': -1}
    df['tono_numerico'] = df['tono'].map(label_mapping)

    # Filtrar filas inválidas
    df = df[df['resumen'].notna() & df['resumen'].apply(lambda x: isinstance(x, str))]
    df = df[df['tono_numerico'].notna()]

    return df

def entrenar_modelo(df):
    X_train, X_test, y_train, y_test = train_test_split(df['resumen'], df['tono_numerico'], test_size=0.2, random_state=42)

    model = make_pipeline(CountVectorizer(min_df=1), LogisticRegression())
    model.fit(X_train, y_train)

    # Evaluar el modelo
    predictions = model.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, predictions)
    print(f'Accuracy: {accuracy:.2f}')

    return model

def analizar_nuevo_dataset(model, archivo):
    df_new = pd.read_excel(archivo)

    # Procesar fila por fila
    resultados = []
    for idx, row in df_new.iterrows():
        if isinstance(row['resumen'], str) and pd.notna(row['resumen']):
            prediccion = model.predict([row['resumen']])[0]
            tono_predicho = {1: 'Positivo', 0: 'Neutro', -1: 'Negativo'}.get(prediccion, 'Desconocido')
        else:
            prediccion = np.nan
            tono_predicho = 'Inválido'

        resultados.append({
            'resumen': row['resumen'],
            'tono_numerico_predicho': prediccion,
            'tono_predicho': tono_predicho
        })

    return pd.DataFrame(resultados)

# Cargar y preprocesar datos de entrenamiento
print("Subir archivo de entrenamiento:")
uploaded_file = files.upload()
file_name = next(iter(uploaded_file))
df_train = cargar_y_preprocesar_datos(file_name)

print(f"Tamaño del dataset de entrenamiento: {len(df_train)}")

# Entrenar el modelo
modelo = entrenar_modelo(df_train)


In [ ]:
# @title Subir excel con nuevas notas (Concatenar Título y resumen)
# Analizar nuevo dataset
print("\nSubir archivo para análisis:")
uploaded_new_data = files.upload()
new_data_file_name = next(iter(uploaded_new_data))
df_resultados = analizar_nuevo_dataset(modelo, new_data_file_name)

# Guardar resultados
output_file_path = 'resultado_sentimiento.xlsx'
df_resultados.to_excel(output_file_path, index=False)
print(f"\nResultados guardados en: {output_file_path}")

# Mostrar los primeros resultados
print("\nPrimeros resultados:")
print(df_resultados.head())

---

##Tema

In [ ]:
# @title Instalación
!pip install --quiet --upgrade fasttext nltk openpyxl tqdm joblib

In [ ]:
# @title Importaciones y utilidades comunes
import os, shutil, re
from pathlib import Path

import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

import nltk
from nltk.corpus import stopwords

# --- Configurar NLTK en un dir. temporal (evita warnings en Colab) ---
nltk_data_dir = '/tmp/nltk_data'
shutil.rmtree(nltk_data_dir, ignore_errors=True)
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path = [nltk_data_dir]
nltk.download('stopwords', download_dir=nltk_data_dir)

# --- Recursos para el pre-procesado ---
stop_words = set(stopwords.words('spanish'))
token_pattern = re.compile(r"\b\w+\b", flags=re.UNICODE)   # tokenización rápida por regex

def preprocess(text: str) -> str:
    if not isinstance(text, str):
        return ""
    tokens = token_pattern.findall(text.lower())
    return " ".join(tok for tok in tokens if tok not in stop_words)

def preprocess_series(serie, desc="Procesando") -> pd.Series:
    """Pre-procesa una Serie de textos en paralelo (tqdm + joblib)."""
    tqdm.pandas(desc=desc)
    n_jobs = -1   # usa todos los núcleos disponibles
    processed = Parallel(n_jobs=n_jobs, backend="loky")(
        delayed(preprocess)(txt) for txt in tqdm(serie, desc=desc)
    )
    return pd.Series(processed, index=serie.index)


In [ ]:
# @title Cargar/limpiar datos de entrenamiento
def preprocess_series(series, desc="Procesando"):
    from tqdm.notebook import tqdm  # o simplemente `from tqdm import tqdm`
    tqdm.pandas(desc=desc)

    # Ajusta esta función a lo que necesites hacer por resumen
    def procesar_texto(texto):
        # Ejemplo de pre-procesamiento (ajústalo a tus necesidades)
        texto = texto.lower()
        texto = texto.strip()
        return texto

    return series.progress_apply(procesar_texto)


In [ ]:
# @title Cargar y evidencia el xlsx de entrenamiento
from pathlib import Path
import pandas as pd

train_path = Path("nissan entreno.xlsx")
use_cols   = ["resumen", "tema"]

df = pd.read_excel(train_path, usecols=use_cols, engine="openpyxl")
df["resumen_procesado"] = preprocess_series(df["resumen"], desc="Tokenizando entreno")

print("✔ Entreno cargado y pre-procesado — filas:", len(df))


In [ ]:
# @title Entrenamiento del modelo
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# --- División train/test ---
X_train_txt, X_test_txt, y_train, y_test = train_test_split(
    df["resumen_procesado"], df["tema"],
    test_size=0.20, random_state=42, stratify=df["tema"]
)

# --- Vectorizador y Naive Bayes ---
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train_txt)
X_test  = vectorizer.transform(X_test_txt)

model = MultinomialNB()
model.fit(X_train, y_train)

# --- Evaluación ---
pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))


In [ ]:
# @title Guardar artefactos para producción
import joblib

joblib.dump(model,      "modelo_naive_bayes.pkl")
joblib.dump(vectorizer, "vectorizador.pkl")
print("✔ Modelo y vectorizador guardados en disco")


In [ ]:
# @title Predicción sobre nuevo lote de noticias
# --- Cargar artefactos guardados ---
loaded_model      = joblib.load("modelo_naive_bayes.pkl")
loaded_vectorizer = joblib.load("vectorizador.pkl")

# --- Cargar archivo de prueba ---
test_path = Path("nissan test.xlsx")
nuevas_df = pd.read_excel(test_path, engine="openpyxl")

# --- Pre-procesar ---
nuevas_df["resumen_procesado"] = preprocess_series(
    nuevas_df["resumen"], desc="Tokenizando test"
)

# --- Vectorizar y predecir ---
X_new = loaded_vectorizer.transform(nuevas_df["resumen_procesado"])
nuevas_df["tema_predicho"] = loaded_model.predict(X_new)

# --- Exportar resultado ---
out_path = "resultado_temas.xlsx"
nuevas_df.to_excel(out_path, index=False)
print(f"✔ Archivo '{out_path}' guardado con {len(nuevas_df)} filas")


---